# Práctica Calificada 2
#### CC421 Inteligencia Artificial

En el presente trabajo buscaremos la manera de 

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from joblib import Parallel, delayed
%matplotlib inline
%config IPCompleter.greedy = True

#### Cargaremos el DataFrame

Debido a que se cuenta con los requerimientos de hardware para trabajar con el DataFrame de manera directa, se usará la librería Pandas para cargar la data.

In [3]:
df = pd.read_csv('train.csv')
df = df.drop(['key'], axis=1)

In [4]:
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [5]:
df.tail()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
55423851,14.0,2014-03-15 03:28:00 UTC,-74.005272,40.740027,-73.963280,40.762555,1
55423852,4.2,2009-03-24 20:46:20 UTC,-73.957784,40.765530,-73.951640,40.773959,1
55423853,14.1,2011-04-02 22:04:24 UTC,-73.970505,40.752325,-73.960537,40.797342,1
55423854,28.9,2011-10-26 05:57:51 UTC,-73.980901,40.764629,-73.870605,40.773963,1
55423855,7.5,2014-12-12 11:33:00 UTC,-73.969722,40.797668,-73.970885,40.783313,1


## Exploración de la data

### Primer tratamiento
#### Búsqueda y saneamiento de valores NaN
   Comprobaremos la existencia de valores nulos para realizar su tratamiento. 

In [6]:
df.isnull().values.any()

True

   Descubriremos qué columnas presentan valores NaN.

In [7]:
df.isnull().any()

fare_amount          False
pickup_datetime      False
pickup_longitude     False
pickup_latitude      False
dropoff_longitude     True
dropoff_latitude      True
passenger_count      False
dtype: bool

Debido a que se encontraron valores NaN en el DataFrame, es necesario saber cuántos son para elegir eliminar o subsanar la data afectada.

   Buscaremos cuántas filas ocupan un valor NaN por columna

In [8]:
df.isnull().sum()

fare_amount            0
pickup_datetime        0
pickup_longitude       0
pickup_latitude        0
dropoff_longitude    376
dropoff_latitude     376
passenger_count        0
dtype: int64

Para visualizar los datos, crearemos un dataframe en el que colocaremos los valores NaN del dataframe df.

In [9]:
nan_rows = df[df.isnull().any(1)]

In [10]:
nan_rows.isnull()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
120227,False,False,False,False,True,True,False
245696,False,False,False,False,True,True,False
340533,False,False,False,False,True,True,False
428108,False,False,False,False,True,True,False
471472,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...
54679078,False,False,False,False,True,True,False
54827432,False,False,False,False,True,True,False
54897759,False,False,False,False,True,True,False
54964682,False,False,False,False,True,True,False


   Del cuadro anterior identificamos que hay 376 filas con valores NaN en sus puntos de llegada; es decir, no se almacenaron datos de llegada.

Como la data afectada es despreciable respecto a la data total, procederemos a retirar dicha data innecesaria.

In [11]:
df = df.dropna(how='any')

Comprobamos que no haya un valor NaN en nuestro DataFrame.

In [12]:
df.isnull().any()

fare_amount          False
pickup_datetime      False
pickup_longitude     False
pickup_latitude      False
dropoff_longitude    False
dropoff_latitude     False
passenger_count      False
dtype: bool

#### Comprobación de los valores de las características
   Aplicaremos los métodos min() y max() para darnos una idea de qué valores toma nuestro DataFrame y pensar en maneras de limpiar la data

In [13]:
df.min()

fare_amount                           -300.0
pickup_datetime      2009-01-01 00:00:27 UTC
pickup_longitude                -3442.059565
pickup_latitude                 -3492.263768
dropoff_longitude               -3442.024565
dropoff_latitude                -3547.886698
passenger_count                            0
dtype: object

In [14]:
df.max()

fare_amount                         93963.36
pickup_datetime      2015-06-30 23:59:54 UTC
pickup_longitude                 3457.625683
pickup_latitude                  3408.789565
dropoff_longitude                 3457.62235
dropoff_latitude                 3537.132528
passenger_count                          208
dtype: object

En los resultados previos se observa incoherencia en la data:

   - Montos de costo negativo y montos excesivos.
        
   - Coordenadas geográficas incoherentes:
    
     ☺  La latitud varía de: -90° a 90°
        
     ☺  La longitud varía de: -180° a 180°
        
   - Si la data consta de viajes en taxi, es improbable que un taxi lleve 208 pasajeros.

### Segundo tratamiento
#### Búsqueda y contabilización de valores incongruentes de Latitud y Longitud

In [15]:
#creo la condicion sobre la columna del dataframe
# platgood = df.loc[:,'pickup_latitude']< -90
#busco aquellas filas del dataframe donde se aplique la condición
#df_mc = np.nan
df=df.drop(df[df['pickup_latitude']< -90].index)
df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...
55423851,14.0,2014-03-15 03:28:00 UTC,-74.005272,40.740027,-73.963280,40.762555,1
55423852,4.2,2009-03-24 20:46:20 UTC,-73.957784,40.765530,-73.951640,40.773959,1
55423853,14.1,2011-04-02 22:04:24 UTC,-73.970505,40.752325,-73.960537,40.797342,1
55423854,28.9,2011-10-26 05:57:51 UTC,-73.980901,40.764629,-73.870605,40.773963,1


In [16]:
df=df.drop(df[df['pickup_latitude']> 90].index)
df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...
55423851,14.0,2014-03-15 03:28:00 UTC,-74.005272,40.740027,-73.963280,40.762555,1
55423852,4.2,2009-03-24 20:46:20 UTC,-73.957784,40.765530,-73.951640,40.773959,1
55423853,14.1,2011-04-02 22:04:24 UTC,-73.970505,40.752325,-73.960537,40.797342,1
55423854,28.9,2011-10-26 05:57:51 UTC,-73.980901,40.764629,-73.870605,40.773963,1


In [17]:
df=df.drop(df[df['dropoff_latitude']> 90].index)
df=df.drop(df[df['dropoff_latitude']< -90].index)
df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...
55423851,14.0,2014-03-15 03:28:00 UTC,-74.005272,40.740027,-73.963280,40.762555,1
55423852,4.2,2009-03-24 20:46:20 UTC,-73.957784,40.765530,-73.951640,40.773959,1
55423853,14.1,2011-04-02 22:04:24 UTC,-73.970505,40.752325,-73.960537,40.797342,1
55423854,28.9,2011-10-26 05:57:51 UTC,-73.980901,40.764629,-73.870605,40.773963,1


In [18]:
df=df.drop(df[df['pickup_longitude']< -180].index)
df=df.drop(df[df['pickup_longitude']> 180].index)
df=df.drop(df[df['dropoff_longitude']< -180].index)
df=df.drop(df[df['dropoff_longitude']> 180].index)

In [19]:
df=df.drop(df[df['fare_amount']<= 0.0].index)

In [20]:
df.max()

fare_amount                         93963.36
pickup_datetime      2015-06-30 23:59:54 UTC
pickup_longitude                  169.972765
pickup_latitude                    89.742163
dropoff_longitude                 169.972765
dropoff_latitude                   89.816665
passenger_count                          208
dtype: object

In [21]:
df=df.drop(df[df['passenger_count']> 10].index)
df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...
55423851,14.0,2014-03-15 03:28:00 UTC,-74.005272,40.740027,-73.963280,40.762555,1
55423852,4.2,2009-03-24 20:46:20 UTC,-73.957784,40.765530,-73.951640,40.773959,1
55423853,14.1,2011-04-02 22:04:24 UTC,-73.970505,40.752325,-73.960537,40.797342,1
55423854,28.9,2011-10-26 05:57:51 UTC,-73.980901,40.764629,-73.870605,40.773963,1


In [22]:
ceros = df.loc[:,'pickup_longitude'] == 0.0
abc = df.loc[ceros]
print(abc.shape)
ceros2 = df.loc[:,'pickup_latitude'] == 0.0
abc = df.loc[ceros2]
print(abc.shape)
ceros3 = df.loc[:,'dropoff_longitude'] == 0.0
abc = df.loc[ceros3]
print(abc.shape)
ceros4 = df.loc[:,'dropoff_latitude'] == 0.0
abc = df.loc[ceros4]
print(abc.shape)

print((df.loc[ceros & ceros2]).shape)

print((df.loc[ceros3 & ceros4]).shape)

print((df.loc[ceros & ceros2 & ceros3 & ceros4]).shape)

(1055224, 7)
(1051745, 7)
(1052043, 7)
(1049032, 7)
(1051455, 7)
(1048754, 7)
(1003078, 7)


In [23]:
ceros5 = df.loc[:,'dropoff_latitude'] != 0.0
print((df.loc[ceros & ceros2 & ceros3 & ceros5]).max())

fare_amount                             98.5
pickup_datetime      2013-09-30 08:46:00 UTC
pickup_longitude                         0.0
pickup_latitude                          0.0
dropoff_longitude                        0.0
dropoff_latitude                   73.984412
passenger_count                            6
dtype: object


 Donde pickup_longitude es 0 = 1055224 rows × 7 columns
 
 Donde pickup_latitude es 0 = 1051745 rows × 7 columns
 
 dropoff_longitude 1052043
 
 dropoff_latitude 1049032
 
 pickup_longitude & pickup_latitude 1051455
 
 dropoff_longitude & dropoff_latitude 1048754
 
 Las 4 condiciones a la vez = 1003078
 

Lo que se procederá a realizar será retirar la data que evidentemente es errada como el tener latitud y longitud igual a cero tanto en puntos de partida como de llegada.

In [24]:
df=df.drop(df[ (df['pickup_longitude']==0.0) &
              (df['pickup_latitude']==0.0) &
              (df['dropoff_longitude']==0.0) &
              (df['dropoff_latitude']==0.0) ].index)

In [25]:
ceros = df.loc[:,'pickup_longitude'] == 0.0
abc = df.loc[ceros]
print(abc.shape)
ceros2 = df.loc[:,'pickup_latitude'] == 0.0
abc = df.loc[ceros2]
print(abc.shape)
ceros3 = df.loc[:,'dropoff_longitude'] == 0.0
abc = df.loc[ceros3]
print(abc.shape)
ceros4 = df.loc[:,'dropoff_latitude'] == 0.0
abc = df.loc[ceros4]
print(abc.shape)


(52146, 7)
(48667, 7)
(48965, 7)
(45954, 7)


La data restante es despreciable pues representa menos del 1% de la data total por lo que para evitar tener datos que aumenten el error, las eliminaremos.

In [26]:
df=df.drop(df[ (df['pickup_longitude']==0.0) |
              (df['pickup_latitude']==0.0) |
              (df['dropoff_longitude']==0.0) |
              (df['dropoff_latitude']==0.0) ].index)

In [27]:
ceros = df.loc[:,'pickup_longitude'] == 0.0
abc = df.loc[ceros]
print(abc.shape)
ceros2 = df.loc[:,'pickup_latitude'] == 0.0
abc = df.loc[ceros2]
print(abc.shape)
ceros3 = df.loc[:,'dropoff_longitude'] == 0.0
abc = df.loc[ceros3]
print(abc.shape)
ceros4 = df.loc[:,'dropoff_latitude'] == 0.0
abc = df.loc[ceros4]
print(abc.shape)

(0, 7)
(0, 7)
(0, 7)
(0, 7)


Ahora ya no tenemos datos en los que el pickup o el dropoff sea 0 en latitud y longitud. Si bien al interseccionar datos se observa que podrían haber tanto datos reales como datos irreales, se prefirió eliminar la data que podría causar errores en el entrenamiento y en la predicción.

In [43]:
poco01 = df.loc[:,'fare_amount'] < 1
test = df.loc[poco01]
test

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
2780,0.01,2015-05-01 15:38:41 UTC,-73.939041,40.713963,-73.941673,40.713997,1
90010,0.01,2015-02-19 06:18:50 UTC,-73.980331,40.770535,-73.981911,40.771503,1
194169,0.01,2015-02-27 15:16:50 UTC,-73.890549,40.719757,-73.842293,40.668804,2
217967,0.01,2015-01-14 16:43:56 UTC,-73.997398,40.725941,-73.997421,40.725983,1
281142,0.01,2015-04-13 21:23:37 UTC,-73.757896,40.927608,-73.757896,40.927605,1
...,...,...,...,...,...,...,...
55073304,0.11,2010-03-22 22:37:00 UTC,-73.793512,40.656922,-73.793508,40.656903,1
55107758,0.01,2015-06-14 21:04:40 UTC,-73.122307,40.787441,-73.122307,40.787445,2
55139906,0.01,2015-06-01 21:46:57 UTC,-73.996986,40.689171,-73.996986,40.689171,1
55157009,0.01,2013-08-25 15:24:22 UTC,-74.177632,40.695360,-74.177622,40.695375,1


Aunque a primera vista, el filtrado anterior pareciese mostrar data errada, podemos despreciarlo porque la cantidad es muy poca para la cantidad de data total.

In [44]:
mucho01 = df.loc[:,'fare_amount'] > 1000
test = df.loc[mucho01]
test

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
18592729,2010.90,2010-03-21 10:03:32 UTC,-73.986265,40.774846,-73.971539,40.743949,1
18859340,61550.86,2013-08-20 11:52:52 UTC,-73.967711,40.755965,-73.945572,40.747666,1
22703574,3130.30,2015-03-09 15:28:56 UTC,-74.005417,40.664413,-74.005417,40.664413,1
30823273,2004.50,2010-03-20 12:24:52 UTC,-73.963134,40.777854,-73.975432,40.782148,1
37879397,2023.70,2010-03-21 14:43:02 UTC,-73.790056,40.647064,-73.917979,40.656940,1
39374511,9000.41,2010-02-19 17:10:00 UTC,-73.993745,40.746105,-73.989807,40.738977,1
41032263,93963.36,2015-02-24 08:57:37 UTC,-73.994461,40.712353,-74.012543,40.702904,2
41186018,75747.02,2015-05-03 09:35:20 UTC,-73.967949,40.792355,-73.990898,40.738976,1
45003632,1564.50,2010-02-20 17:38:00 UTC,-73.937783,40.758255,-73.937748,40.758232,1


Observamos que también existen costos elevados. Para analizar si debería ser correcto o no el costo, es importante analizar el motivo del costo, si no tiene fundamento se considerará como data incorrecta y deberá ser eliminada.

### Ingeniería de características

#### Pautas
Hemos optado por crear una característica que representará la distancia del trayecto, la cuál supone que los trayectos se realizan en una superficie plana. 

In [45]:
#df.loc[:,['a','c']]
datox1=df.loc[:,'pickup_longitude']
datox2=df.loc[:,'dropoff_longitude']
datoy1=df.loc[:,'pickup_latitude']
datoy2=df.loc[:,'dropoff_latitude']

#df.apply(np.sqrt)
datoX=(datox1-datox2)**2
datoY=(datoy1-datoy2)**2

print(datoX)
print(datoY)



0           0.000007
1           0.001353
2           0.000072
3           0.000020
4           0.000131
              ...   
55423851    0.001763
55423852    0.000038
55423853    0.000099
55423854    0.012165
55423855    0.000001
Length: 54313237, dtype: float64
0           0.000082
1           0.004999
2           0.000115
3           0.000622
4           0.000248
              ...   
55423851    0.000508
55423852    0.000071
55423853    0.002027
55423854    0.000087
55423855    0.000206
Length: 54313237, dtype: float64


In [59]:
datoFinal=((datoX+datoY).apply(np.sqrt))*10000
print(datoFinal)

0             94.358403
1            796.956699
2            136.740367
3            253.404730
4            194.695176
               ...     
55423851     476.533194
55423852     104.305694
55423853     461.073889
55423854    1106.902488
55423855     144.020344
Length: 54313237, dtype: float64


Se ha multiplicado por 10000 solo para facilitar la legibilidad.

In [60]:
import warnings
warnings.filterwarnings('ignore')

df.loc[:,'distance'] = datoFinal
print(df)

          fare_amount          pickup_datetime  pickup_longitude  \
0                 4.5  2009-06-15 17:26:21 UTC        -73.844311   
1                16.9  2010-01-05 16:52:16 UTC        -74.016048   
2                 5.7  2011-08-18 00:35:00 UTC        -73.982738   
3                 7.7  2012-04-21 04:30:42 UTC        -73.987130   
4                 5.3  2010-03-09 07:51:00 UTC        -73.968095   
...               ...                      ...               ...   
55423851         14.0  2014-03-15 03:28:00 UTC        -74.005272   
55423852          4.2  2009-03-24 20:46:20 UTC        -73.957784   
55423853         14.1  2011-04-02 22:04:24 UTC        -73.970505   
55423854         28.9  2011-10-26 05:57:51 UTC        -73.980901   
55423855          7.5  2014-12-12 11:33:00 UTC        -73.969722   

          pickup_latitude  dropoff_longitude  dropoff_latitude  \
0               40.721319         -73.841610         40.712278   
1               40.711303         -73.979268       

In [61]:
df.min()

fare_amount                             0.01
pickup_datetime      2009-01-01 00:00:27 UTC
pickup_longitude                 -168.603534
pickup_latitude                   -77.833874
dropoff_longitude                 -173.95763
dropoff_latitude                  -74.354612
passenger_count                            0
distance                                0.01
dtype: object

Analicemos el caso en que no hay pasajeros, podríamos tomar los datos como referencia de "delivery".

In [62]:
psj01 = df.loc[:,'passenger_count'] ==0
test = df.loc[psj01]
test

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
314,34.0,2015-06-02 23:16:15 UTC,-73.974899,40.751095,-73.908546,40.881878,0,1466.523584
566,4.9,2012-01-28 21:33:18 UTC,-73.955322,40.782840,-73.955797,40.773673,0,91.792981
678,6.5,2012-02-27 07:24:20 UTC,-73.983397,40.738183,-73.971395,40.758023,0,231.877900
1160,13.3,2011-05-25 23:58:48 UTC,-73.998360,40.740348,-73.946455,40.777348,0,637.426782
1935,10.1,2011-10-23 11:09:28 UTC,-73.971400,40.795000,-73.967900,40.768600,0,266.309970
...,...,...,...,...,...,...,...,...
55422287,17.7,2011-07-31 15:25:05 UTC,-73.975100,40.681000,-73.980300,40.742400,0,616.198020
55422918,11.7,2011-10-19 18:46:09 UTC,-73.985200,40.755900,-73.972000,40.764700,0,158.644256
55422965,6.9,2012-02-08 23:52:37 UTC,-74.004172,40.751831,-73.994318,40.738853,0,162.950851
55422985,6.5,2011-10-21 10:21:32 UTC,-73.982300,40.768100,-73.968200,40.760400,0,160.654910


Si bien podemos tolerar la inexistencia de pasajeros, no podemos tolerar el hecho de que se cobre por una distancia recorrida igual a cero, es incoherente.

In [63]:
dst01 = df.loc[:,'distance'] ==0
test = df.loc[dst01]
test

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance


In [64]:
df=df.drop(df[df['distance'] == 0].index)
df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,94.358403
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,796.956699
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,136.740367
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,253.404730
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,194.695176
...,...,...,...,...,...,...,...,...
55423851,14.0,2014-03-15 03:28:00 UTC,-74.005272,40.740027,-73.963280,40.762555,1,476.533194
55423852,4.2,2009-03-24 20:46:20 UTC,-73.957784,40.765530,-73.951640,40.773959,1,104.305694
55423853,14.1,2011-04-02 22:04:24 UTC,-73.970505,40.752325,-73.960537,40.797342,1,461.073889
55423854,28.9,2011-10-26 05:57:51 UTC,-73.980901,40.764629,-73.870605,40.773963,1,1106.902488


También podemos encontrar datos en donde por una distancia mínima se realiza un cobro anormal.

In [66]:
dst02 = df.loc[:,'distance'] <=0.1
fare01 = df.loc[:,'fare_amount'] > 10
test = df.loc[dst02 & fare01]
test

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
510,49.57,2010-10-23 18:39:00 UTC,-73.782215,40.644272,-73.782217,40.644270,2,0.028284
4861,49.80,2012-02-05 11:08:07 UTC,-73.988168,40.745863,-73.988172,40.745870,1,0.080623
15093,45.00,2010-11-07 20:17:00 UTC,-73.782380,40.648773,-73.782375,40.648767,1,0.078102
15366,50.50,2010-05-22 13:38:44 UTC,-73.776343,40.645167,-73.776342,40.645169,1,0.022361
16342,20.00,2014-06-13 17:41:59 UTC,-74.002628,40.718813,-74.002622,40.718806,1,0.092195
...,...,...,...,...,...,...,...,...
55405084,55.00,2010-08-15 03:17:00 UTC,-73.785947,40.923437,-73.785948,40.923438,1,0.014142
55414812,100.00,2009-12-21 20:06:00 UTC,-73.414340,40.762877,-73.414343,40.762875,1,0.036056
55415176,79.00,2013-11-20 22:33:39 UTC,-73.979538,40.851751,-73.979531,40.851749,1,0.072801
55416114,19.50,2014-10-13 01:25:14 UTC,-73.994214,40.690240,-73.994208,40.690242,1,0.063246


In [68]:
df=df.drop(df[ (df['distance'] <=0.1) & (df['fare_amount'] >10) ].index)
df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,94.358403
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,796.956699
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,136.740367
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,253.404730
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,194.695176
...,...,...,...,...,...,...,...,...
55423851,14.0,2014-03-15 03:28:00 UTC,-74.005272,40.740027,-73.963280,40.762555,1,476.533194
55423852,4.2,2009-03-24 20:46:20 UTC,-73.957784,40.765530,-73.951640,40.773959,1,104.305694
55423853,14.1,2011-04-02 22:04:24 UTC,-73.970505,40.752325,-73.960537,40.797342,1,461.073889
55423854,28.9,2011-10-26 05:57:51 UTC,-73.980901,40.764629,-73.870605,40.773963,1,1106.902488


In [ ]:
#https://github.com/jmcarpenter2/swifter
import swifter
df['pickup_datetime']=df['pickup_datetime'].swifter.apply(pd.to_datetime)

In [69]:
ruta = "/IA PC2/GitHub01/Practica2/NuevoCSV2.csv"
df.to_csv(ruta, index=False)